In [204]:
from main import Solver
import torch
import numpy as np
import pandas as pd

# Hyper Params Section
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using", torch.cuda.device_count(), "GPUs")


Exp_label = pd.read_csv('Exp664_genes.txt', sep='\t',header = None)
Exp_list = Exp_label.iloc[:, 0].values.tolist()
Exp = pd.DataFrame(np.loadtxt("Exp664.txt").T,columns = Exp_label)

Meth_label = pd.read_csv('Meth664_probes.txt', sep='\t',header = None)
Meth_list = Meth_label.iloc[:, 0].values.tolist()
Meth = pd.DataFrame(np.loadtxt("Meth664.txt").T,columns = Meth_label)

miRNA_label = pd.read_csv('miRNA664_miRNA.txt', sep='\t',header = None)
miRNA_list = miRNA_label.iloc[:, 0].values.tolist()
miRNA = pd.DataFrame(np.loadtxt("miRNA664.txt").T,columns = miRNA_label)

y = pd.read_csv('PAM50label664.txt',header = None)
views = [Exp,Meth,miRNA]


length = [1]

for i, view in enumerate(views):
    length.append(view.shape[1])

## Merging 3 Inputs
Merge = pd.concat((Exp, Meth, miRNA), axis=1)

from sklearn.model_selection import train_test_split
## Train 0.6, Val 0.2, Test 0.2
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

size_1 = test_ratio
size_2 = val_ratio / (train_ratio+val_ratio)

# Train, Test
X_train, X_test, y_train, y_test = train_test_split(Merge, y, test_size=size_1, random_state=42)
# Train, Val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=size_2, random_state=42)

train_dataframe = []
val_dataframe = []
test_dataframe = []
train_data = []
val_data = []
test_data = []
print(X_train.shape)
for i in range(len(length)-1):
    a = sum(length[:(i+1)])
    b = sum(length[:(i+2)])
    train_dataframe.append(X_train.iloc[:,a:b])
    train_data.append(torch.tensor(X_train.iloc[:,a:b].values))
    val_dataframe.append(X_val.iloc[:,a:b])
    val_data.append(torch.tensor(X_val.iloc[:,a:b].values))
    test_dataframe.append(X_test.iloc[:,a:b])
    test_data.append(torch.tensor(X_test.iloc[:,a:b].values))

print(f'input train data shape :')
for i, view in enumerate(train_data):
    print(f'view_{i} :  {view.shape}')
    view = view.to(device)
print(f'input validation data shape :')
for i, view in enumerate(val_data):
    print(f'view_{i} :  {view.shape}')
    view = view.to(device)
print(f'input test data shape :')
for i, view in enumerate(test_data):
    print(f'view_{i} :  {view.shape}')
    view = view.to(device)
print(f'input label data shape :',y_train.shape,y_val.shape,y_test.shape)

Solver = Solver(device)

Using 1 GPUs
(398, 6377)
input train data shape :
view_0 :  torch.Size([398, 2642])
view_1 :  torch.Size([398, 3298])
view_2 :  torch.Size([398, 436])
input validation data shape :
view_0 :  torch.Size([133, 2642])
view_1 :  torch.Size([133, 3298])
view_2 :  torch.Size([133, 436])
input test data shape :
view_0 :  torch.Size([133, 2642])
view_1 :  torch.Size([133, 3298])
view_2 :  torch.Size([133, 436])
input label data shape : (398, 1) (133, 1) (133, 1)


In [ ]:
## Analysis

u = Solver._get_outputs(train_data, 1e-7, 100, (1,1,1))


SNGCCA Started!
